# GOMP0114 Inverse Problems in Imaging. Coursework 2
### Student ID: 18145399
## Week 1 
### 1. Convolution and deconvolution


(a) Read a gray colormap image from the given URL and convert it to a float, normalise and display it.

![solution](1a.png)

(b)  Write a function that takes in an image $f$ and outputs the blurred image $Af$ with convolution mapping.
$$g = Af_{true} + n$$

In [ ]:
import scipy.ndimage.filters as filters

def apply_convolution(f, sigma, theta):
    # Apply Gaussian filter
    g = filters.gaussian_filter(f, sigma)
    
    # Add noise to blurred image
    w, h = g.shape
    noise = np.random.randn(w, h)
    g = g + theta * noise
    
    return g  

(c) Deconvolve using normal equations, i.e. find $f_α$ as the solution to
$$(A^TA + αI)f_α = A^Tg$$

In [ ]:
import scipy.sparse as sparse
import scipy.sparse.linalg as splinalg

def ATA_operator(f, sigma, alpha):
    # Apply A^T A + alpha*I operator to f
    Af = filters.gaussian_filter(f, sigma) 
    ATAf = filters.gaussian_filter(Af, sigma)
    return ATAf + alpha * f

def ATA(f, sigma, alpha):
    # Compute A.TA(f) using Gaussian convolution
    Af = filters.gaussian_filter(f, sigma)
    Af = Af.reshape(-1, 1)
    ATAf = filters.gaussian_filter(Af.reshape(f.shape), sigma)
    ATAf = (ATAf + alpha * f).reshape(-1)
    return ATAf

def deconvolve_normal_equations(g, sigma, alpha):
    # Set up linear operator for ATA
    M, N = g.shape
    A = sparse.linalg.LinearOperator((M*N, M*N), matvec=lambda x: np.ravel(ATA_operator(x.reshape(g.shape), sigma, alpha)))

    # Compute ATg
    ATg = np.ravel(g)

    # Solve linear system using GMRES
    f_alpha, info = splinalg.gmres(A, ATg)

    return f_alpha.reshape((M, N))

def normal_info(g, sigma, alpha):
    # Set up linear operator for ATA
    M, N = g.shape
    A = sparse.linalg.LinearOperator((M*N, M*N), matvec=lambda x: np.ravel(ATA_operator(x.reshape(g.shape), sigma, alpha)))

    # Compute ATg
    ATg = np.ravel(g)

    # Solve linear system using GMRES
    f_alpha, info = splinalg.gmres(A, ATg)

    return info

(d) Deconvolve by solving the augmented equations.
$$
\begin{pmatrix}
A\\
\sqrt{\alpha}I
\end{pmatrix}f 
= 
\begin{pmatrix}
g\\
0
\end{pmatrix}
$$

In [ ]:
def M_f(f):
    # Implementation of the augmented matrix multiplication
    y = filters.gaussian_filter(f, sigma)
    z = filters.gaussian_filter(y, sigma)
    M_f = np.vstack([np.ravel(z), np.sqrt(alpha)*np.ravel(f)])
    return M_f

def MT_b(b):
    # Implementation of the transposed augmented matrix multiplication
    global g
    M, N = g.shape
    g_vec = b[:M*N]
    f_vec = b[M*N:]
    g = np.reshape(g_vec, (M, N))
    y = filters.gaussian_filter(g, sigma)
    z = filters.gaussian_filter(y, sigma)
    MT_b = np.ravel(z) + np.sqrt(alpha)*np.ravel(f_vec)
    return MT_b

def solve_augmented_equations(g, sigma, alpha):
    # Define linear operator for lsqr
    M, N = g.shape
    size = M*N
    A = sparse.linalg.LinearOperator((2*size, size), matvec=M_f, rmatvec=MT_b)

    # Concatenate g with a zero vector
    b = np.vstack([np.reshape(g,(size,1)), np.zeros((size, 1))])
    
    # Solve linear system using lsqr
    f_lsqr= splinalg.lsqr(A, b)[0]

    return f_lsqr[:g.size].reshape(g.shape)

def augmented_info(g, sigma, alpha):
    # Define linear operator for lsqr
    M, N = g.shape
    size = M*N
    A = sparse.linalg.LinearOperator((2*size, size), matvec=M_f, rmatvec=MT_b)

    # Concatenate g with a zero vector
    b = np.vstack([np.reshape(g,(size,1)), np.zeros((size, 1))])
    
    # Solve linear system using lsqr
    info = splinalg.lsqr(A, b)[1]

    return info

Compare the performance to the one used in c.), in terms of number of iterations required to achieve convergence:

Method c: Converged in 0 iterations (0.03956246376037598 seconds) 

Method d: Converged in 2 iterations (0.7001798152923584 seconds)

![solution](1d.png)

![solution](1d2.png)

### Comments:

#### The choice of the value of alpha
When the value of the regularization parameter alpha was set to 0.01, the deblurred image produced by Method c was unclear. However, when alpha was set to 1, the deblurred image became much clearer.

It is possible that alpha=0.01 was too small, and the regularization term had little effect in suppressing the noise in the observed image. As a result, the noise dominated the solution, leading to an unclear deblurred image. On the other hand, alpha=1 may have been a better choice, as it struck a good balance between noise suppression and image fidelity, resulting in a clearer deblurred image.

#### The choice of Deconvolution method 
Method c is faster and requires fewer iterations to converge than Method d.

Method c uses the normal equations, which can be solved using a Krylov solver such as PCG or GMRES. The normal equations can be derived from the optimality conditions for the Tikhonov regularization problem, and their solution provides the optimal solution to the deblurring problem. Since the normal equations involve a symmetric positive definite matrix, a Krylov solver can efficiently solve the linear system without requiring an explicit matrix representation of the convolution operator.

On the other hand, Method d uses an augmented equation approach, which involves solving a linear least squares problem using a solver such as lsqr. This method may require more iterations to converge because the least squares problem may not have a closed-form solution and the iterative solver may need to compute many iterations to approximate the solution. In addition, the implementation of the augmented equation method requires more memory to store the augmented system matrix.

## Week 2:
### 2. Choose a regularisation parameter α
#### i)  Discrepency Principle
Use Discrepency Principle method to choose an optimal value for α for the solution in task 1.

#### ii) L-Curve
Use L-Curve method to choose an optimal value for α for the solution in task 1.

### Comments
Comment on the difference in value obtained and the results using these two values.

### 3. Using a regularisation term based on the spatial derivative
#### 